In [2]:
#Adam Bloodgood

import requests
import json
import pandas as pd
import string
import math
import numpy as np

In [3]:
# Gather details from all of the shots from a specific game ID.

def getShotInfo(play_by_play):

    # Get play-by-play data from the selected game and filter out just the individual plays.

    # Filter out plays that are either shots or goals
    event_types = ["goal", "shot-on-goal"]

    data = pd.DataFrame.from_dict(play_by_play['plays'])
    shots = data[data['typeDescKey'].isin(event_types)]

    shots = shots.reset_index()

    for index, shot in shots.iterrows():
        descriptor = shot['periodDescriptor']
        if descriptor['number'] == 5:
            shots.drop(index, inplace=True)

    # Clean up dataframe to make sense
    shot_details = pd.json_normalize(shots['details'])
    shot_results = shots['typeDescKey'].reset_index()
    homeTeamSide = shots['homeTeamDefendingSide'].reset_index()
    situationCode = shots['situationCode'].reset_index()
    period = shots['periodDescriptor'].reset_index()
    shot_details = pd.concat([shot_results, shot_details, homeTeamSide, period, situationCode], axis=1)
    del shot_details['index']

    return shot_details

def getPlayerName(playerId, rosters):

    #Get player name from current game roster

    for player in rosters:
        if player['playerId'] == playerId:
            first_name = player['firstName']['default']
            last_name = player['lastName']['default']

            return f"{last_name}, {first_name}", player['positionCode']
    return

def getShotCoords(shot, homeTeam):

    #Get coordinates and shot side from shot data

    x_coord = shot['xCoord']
    y_coord = shot['yCoord']
    
    if shot['eventOwnerTeamId'] == homeTeam:
        if shot['homeTeamDefendingSide'] == 'right':
            net_side = 'left'
        else:
            net_side = 'right'
    else:
        net_side = shot['homeTeamDefendingSide']

    return x_coord, y_coord, net_side

def calcAzimuthErrorMargin(x: int, y: int, net_side):

    #calculate margin of error in the azimuth dimension from puck to between posts

    if net_side == 'left':
        left_post = {'x': -89, 'y': -3}
        right_post = {'x': -89, 'y': 3}
    else:
        left_post = {'x': 89, 'y': 3}
        right_post = {'x': 89, 'y': -3}

    if x == left_post['x']:
        if y <= 3 and y >= -3:
            return math.pi / 2, (math.pi / 2) * -1
        else:
            return 0.0, 0.0

    angle_to_left = math.atan((left_post['y'] - y)/(left_post['x'] - x))
    angle_to_right = math.atan((right_post['y'] - y)/(right_post['x'] - x))

    return angle_to_left, angle_to_right

def calcElevationAngles(x: int, y: int, shot_side):

    #calculate the elevation margin of error from the puck to the corners of the crossbar
    #NOT CURRENTLY USED

    if shot_side == 'left':
        left_post = {'x': -89, 'y': -3}
        right_post = {'x': -89, 'y': 3}
    else:
        left_post = {'x': 89, 'y': 3}
        right_post = {'x': 89, 'y': -3}
    
    if x == left_post['x']:
        if y <= 3 and y >= -3:
            return math.pi / 2, math.pi / 2
        
    distance_to_left_post = math.sqrt(((left_post['x'] - x) ** 2) + ((left_post['y'] - y) ** 2))
    distance_to_right_post = math.sqrt(((right_post['x'] - x) ** 2) + ((right_post['y'] - y) ** 2))

    angle_to_left_post = math.atan(4 / distance_to_left_post)

    angle_to_right_post = math.atan(4 / distance_to_right_post)

    return angle_to_left_post, angle_to_right_post

def calcElevationToPoint(x: int, y: int, point_x: int, point_y: int):
    
    #calculate the elevation margin of error from the puck to a point on the crossbar.
    #NOT CURRENTLY USED
    
    if x == point_x:
        if y <= 3 and y >= -3:
            return math.pi / 2
        
    distance_to_point = math.sqrt(((point_x - x) ** 2) + ((point_y - y) ** 2))

    angle_to_point = math.atan(4 / distance_to_point)

    return angle_to_point

def calcSurfaceAreaErrorMargin(az_left, az_right, el_left, el_right):

    #calculate radial surface area margin of error from the puck to the net.
    #NOT CURRENTLY USED
        
    az_margin = az_left - az_right

    area_1 = (az_margin)*(math.cos(0) - math.cos(el_left))
    area_2 = (az_margin)*(math.cos(0) - math.cos(el_right))

    area = (abs(area_1) + abs(area_2)) / 2

    return abs(area)

def calcSurfaceAreaWithinPosts(az_left, az_middle, az_right, el_left, el_middle, el_right):

    #calculate radial surface area margin of error from the puck to the net.
    #NOT CURRENTLY USED

    area1 = calcSurfaceAreaErrorMargin(az_left, az_middle, el_left, el_middle)
    area2 = calcSurfaceAreaErrorMargin(az_middle, az_right, el_middle, el_right)

    area = abs(area1) + abs(area2)

    return abs(area)

def calcShotDifficulty(x, y, side):

    #calculate radial surface area margin of error from the puck to the net.
    #NOT CURRENTLY USED

    if x < -89 or x > 89:
        return 1 
    
    if side == 'left':
        net_x = -89
    else:
        net_x = 89

    az_left, az_right = calcAzimuthErrorMargin(x, y, side)

    el_left, el_right = calcElevationAngles(x, y, side)

    #print(f"{az_left}, {az_right}, {el_left}, {el_right}")

    if y < -3 or y > 3:
        area = calcSurfaceAreaErrorMargin(az_left, az_right, el_left, el_right )
    else:
        el_middle = calcElevationToPoint(x, y, net_x, y)
        #print(f"el_middle: {el_middle}")
        area = calcSurfaceAreaWithinPosts(az_left, 0, az_right, el_left, el_middle, el_right)

    difficulty = (math.pi - area) / math.pi

    return(difficulty)


def replaceIdsWithNames(player_db):

    #find player name based on ID.

    new_db = {}

    for player_id in player_db.keys():
        player_info = requests.get(f"https://api-web.nhle.com/v1/player/{player_id}/landing")

        # Make sure the shot was not on an empty net
        if player_info.status_code != 404:
            player_info = player_info.json()
            
            
            first_name = player_info['firstName']['default']
            last_name = player_info['lastName']['default']

            full_name = f"{last_name}, {first_name}"
            new_db[full_name] = player_db[player_id]

    return new_db

def isPowerPlay(shot, homeTeam):
    
    #Checks if shot taken was on the power play.
    #Situation code abcd. a = away goalie, b = away skaters, c = home skaters, d = home goalie. ex. 1451 = home 5v4 power play
    if shot['eventOwnerTeamId'] == homeTeam:
        if shot['situationCode'] == '1451' or shot['situationCode'] == '1351' or shot['situationCode'] == '1341' or shot['situationCode'] == '1450' or shot['situationCode'] == '1460' or shot['situationCode'] == '1360' or shot['situationCode'] == '1340':
            return True
        return False
    else:
        if shot['situationCode'] == '1541' or shot['situationCode'] == '1531' or shot['situationCode'] == '1431' or shot['situationCode'] == '0541' or shot['situationCode'] == '0641' or shot['situationCode'] == '0631' or shot['situationCode'] == '0431':
            return True
        return False



In [11]:
player_db = {}
goalie_db = {}

SEASON = 2023
SEASON_TYPE = 2 # 1 = preseason, 2 = regular season, 3 = playoffs
GAMES_TRACKING = 1312
START_GAME = 1

for game in range(START_GAME, START_GAME + GAMES_TRACKING):
    print(game)
    
    # Insert appropriate number of zeros to get the right play-by-play url
    game_code = game
    if game < 10:
        game_code = f"000{game}"
    elif game < 100:
        game_code = f"00{game}"
    elif game < 1000:
        game_code = f"0{game}"

    gameId = f"{SEASON}0{SEASON_TYPE}{game_code}"

    play_by_play = requests.get(f"https://api-web.nhle.com/v1/gamecenter/{gameId}/play-by-play")
    play_by_play = play_by_play.json()

    home_team = play_by_play['homeTeam']['id']
    rosters = play_by_play['rosterSpots']

    shot_details = getShotInfo(play_by_play)

    shots = 0
    for index, shot in shot_details.iterrows():
        
        if math.isnan(shot['goalieInNetId']) == False:
            x, y, net_side = getShotCoords(shot, home_team)
            isPP = isPowerPlay(shot, home_team)
            player_id = int(shot['goalieInNetId'])
            player_id, position = getPlayerName(player_id, rosters)
            if player_id not in player_db:
                player_db[player_id] = {'position': position, 
                                        'avg. shot difficulty': 0.0, 'avg. goal difficulty': 0.0, 'adjusted sv%': 0.0, 'shot-goal difference': 0.0,
                                        'avg. pp shot difficulty': 0.0, 'avg. pp goal difficulty': 0.0, 'adjusted pp sv%': 0.0, 'pp shot-goal difference': 0.0,
                                        'avg. ev shot difficulty': 0.0, 'avg. ev goal difficulty': 0.0, 'adjusted ev sv%': 0.0, 'ev shot-goal difference': 0.0,
                                        'shots': 0, 'pp shots': 0, 'ev shots': 0, 'goals against': 0, 'pp goals against': 0, 'ev goals against': 0, "sv%": 0.0, 'pp sv%': 0.0, 'ev sv%': 0.0,
                                        'save locations': [], 'pp save locations': [], 'ev save locations': [], 'goal locations': [], 'pp goal locations': [], 'ev goal locations': [],
                                        }
            
            if ((x >= 25 and net_side == 'right') or (x <= -25 and net_side == 'left')):
                if x > 89 or x < -89:
                    shot_difficulty = 1
                else:
                    az_left, az_right = calcAzimuthErrorMargin(x, y, net_side)
                    margin = abs(az_left-az_right)
                    shot_difficulty = (math.pi - margin) / math.pi
                
                #shot_difficulty = calcShotDifficulty(x, y, net_side)

                if shot['typeDescKey'] == "goal":
                    player_db[player_id]['avg. goal difficulty'] = ((player_db[player_id]['avg. goal difficulty'] *
                                                                    player_db[player_id]['goals against'] + shot_difficulty) /
                                                                    ( player_db[player_id]['goals against'] + 1))
                    player_db[player_id]['goals against'] += 1
                    player_db[player_id]['goal locations'].append([x, y, net_side])
                    if isPP:
                        player_db[player_id]['avg. pp goal difficulty'] = ((player_db[player_id]['avg. pp goal difficulty'] *
                                                                    player_db[player_id]['pp goals against'] + shot_difficulty) /
                                                                    ( player_db[player_id]['pp goals against'] + 1))
                        player_db[player_id]['pp goals against'] += 1
                        player_db[player_id]['pp goal locations'].append([x, y, net_side])
                    else:
                        player_db[player_id]['avg. ev goal difficulty'] = ((player_db[player_id]['avg. ev goal difficulty'] *
                                                                    player_db[player_id]['ev goals against'] + shot_difficulty) /
                                                                    ( player_db[player_id]['ev goals against'] + 1))
                        player_db[player_id]['ev goals against'] += 1
                        player_db[player_id]['ev goal locations'].append([x, y, net_side])
                else:
                    player_db[player_id]['save locations'].append([x, y, net_side])
                    if isPP:
                        player_db[player_id]['pp save locations'].append([x, y, net_side])
                    else:
                        player_db[player_id]['ev save locations'].append([x, y, net_side])

                player_db[player_id]['avg. shot difficulty'] = ((player_db[player_id]['avg. shot difficulty'] *
                                                                    player_db[player_id]['shots'] + shot_difficulty) /
                                                                    ( player_db[player_id]['shots'] + 1))
                player_db[player_id]['shots'] += 1
                if isPP:
                    player_db[player_id]['avg. pp shot difficulty'] = ((player_db[player_id]['avg. pp shot difficulty'] *
                                                                    player_db[player_id]['pp shots'] + shot_difficulty) /
                                                                    ( player_db[player_id]['pp shots'] + 1))
                    player_db[player_id]['pp shots'] += 1
                else:
                    player_db[player_id]['avg. ev shot difficulty'] = ((player_db[player_id]['avg. ev shot difficulty'] *
                                                                    player_db[player_id]['ev shots'] + shot_difficulty) /
                                                                    ( player_db[player_id]['ev shots'] + 1))
                    player_db[player_id]['ev shots'] += 1
                



    

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [12]:

player_db_new = pd.DataFrame.from_dict(player_db, orient='index')
player_db_new

,position,avg. shot difficulty,avg. goal difficulty,adjusted sv%,shot-goal difference,avg. pp shot difficulty,avg. pp goal difficulty,adjusted pp sv%,pp shot-goal difference,avg. ev shot difficulty,...,ev goals against,sv%,pp sv%,ev sv%,save locations,pp save locations,ev save locations,goal locations,pp goal locations,ev goal locations
"Saros, Juuse",G,0.929616,0.881878,0.0,0.0,0.918793,0.885689,0.0,0.0,0.931616,...,134,0.0,0.0,0.0,"[[59, -26, right], [81, 8, right], [55, 30, ri...","[[34, 8, right], [-74, 17, left], [54, 14, rig...","[[59, -26, right], [81, 8, right], [55, 30, ri...","[[50, -16, right], [83, 0, right], [71, 7, rig...","[[83, 0, right], [82, 2, right], [87, -9, righ...","[[50, -16, right], [71, 7, right], [-77, 1, le..."
"Johansson, Jonas",G,0.925172,0.879065,0.0,0.0,0.914152,0.866855,0.0,0.0,0.926966,...,69,0.0,0.0,0.0,"[[-75, -15, left], [-47, -25, left], [72, 39, ...","[[82, 1, right], [75, -16, right], [80, 8, rig...","[[-75, -15, left], [-47, -25, left], [72, 39, ...","[[81, 1, right], [-82, 4, left], [-73, -23, le...","[[-73, -23, left], [-78, 5, left], [47, 8, rig...","[[81, 1, right], [-82, 4, left], [-88, -1, lef..."
"Mrazek, Petr",G,0.926487,0.877889,0.0,0.0,0.905391,0.868150,0.0,0.0,0.929426,...,128,0.0,0.0,0.0,"[[75, 21, right], [32, -8, right], [98, 12, ri...","[[59, -5, right], [60, -3, right], [62, 27, ri...","[[75, 21, right], [32, -8, right], [98, 12, ri...","[[83, -9, right], [-82, -19, left], [85, 5, ri...","[[-88, -4, left], [33, -4, right], [68, 2, rig...","[[83, -9, right], [-82, -19, left], [85, 5, ri..."
"Jarry, Tristan",G,0.928988,0.879448,0.0,0.0,0.921079,0.872667,0.0,0.0,0.930138,...,108,0.0,0.0,0.0,"[[-55, -35, left], [-39, -27, left], [-36, -30...","[[-60, 25, left], [-78, -6, left], [-78, 2, le...","[[-55, -35, left], [-39, -27, left], [-36, -30...","[[83, 0, right], [-71, 11, left], [-67, 11, le...","[[83, -2, right], [-80, -5, left], [72, 21, ri...","[[83, 0, right], [-71, 11, left], [-67, 11, le..."
"Hill, Adin",G,0.931344,0.893113,0.0,0.0,0.920822,0.925136,0.0,0.0,0.933109,...,77,0.0,0.0,0.0,"[[74, 25, right], [87, -5, right], [68, 12, ri...","[[75, -7, right], [67, -6, right], [59, -28, r...","[[74, 25, right], [87, -5, right], [68, 12, ri...","[[-75, -1, left], [72, 6, right], [-86, -4, le...","[[-59, 2, left], [-51, 3, left], [-80, -4, lef...","[[-75, -1, left], [72, 6, right], [-86, -4, le..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Cooley, Devin",G,0.927392,0.882190,0.0,0.0,0.934199,0.940700,0.0,0.0,0.926287,...,24,0.0,0.0,0.0,"[[-62, -18, left], [-78, 1, left], [-76, 13, l...","[[-35, -5, left], [-55, -21, left], [-58, -8, ...","[[-62, -18, left], [-78, 1, left], [-76, 13, l...","[[80, 0, right], [-62, 10, left], [-31, -15, l...","[[-59, -8, left]]","[[80, 0, right], [-62, 10, left], [-31, -15, l..."
"Silovs, Arturs",G,0.932898,0.896574,0.0,0.0,0.946684,0.927495,0.0,0.0,0.929236,...,7,0.0,0.0,0.0,"[[53, 36, right], [30, -35, right], [43, -5, r...","[[-34, -11, left], [-65, 20, left], [-80, 25, ...","[[53, 36, right], [30, -35, right], [43, -5, r...","[[78, 29, right], [82, -4, right], [75, -1, ri...","[[75, -1, right], [52, -11, right], [-39, -11,...","[[78, 29, right], [82, -4, right], [79, 14, ri..."
"Fedotov, Ivan",G,0.931939,0.895528,0.0,0.0,0.961877,0.000000,0.0,0.0,0.931328,...,10,0.0,0.0,0.0,"[[-60, -39, left], [-66, -3, left], [-64, -22,...","[[-54, -23, left]]","[[-60, -39, left], [-66, -3, left], [-64, -22,...","[[-79, -6, left], [-67, 3, left], [86, -8, rig...",[],"[[-79, -6, left], [-67, 3, left], [86, -8, rig..."
"Subban, Malcolm",G,0.924922,0.953617,0.0,0.0,0.883222,0.942035,0.0,0.0,0.935347,...,2,0.0,0.0,0.0,"[[-98, 22, left], [-51, 26, left], [-78, -12, ...","[[82, 5, right], [85, 4, right], [43, 3, right...","[[-98, 22, left], [-51, 26, left], [-78, -12, ...","[[-62, 23, left], [-77, 16, left], [-54, -21, ...","[[-77, 16, left]]","[[-62, 23, left], [-54, -21, left]]"


In [13]:
for player_id in player_db.keys():
    if player_db[player_id]['shots'] == 0:
        player_db[player_id]['sv%'] = 0.0
        player_db[player_id]['pp sv%'] = 0.0
        player_db[player_id]['ev sv%'] = 0.0
    else:
        player_db[player_id]['sv%'] = float(player_db[player_id]['shots'] - player_db[player_id]['goals against']) / float(player_db[player_id]['shots'])
        if player_db[player_id]['pp shots'] != 0: player_db[player_id]['pp sv%'] = float(player_db[player_id]['pp shots'] - player_db[player_id]['pp goals against']) / float(player_db[player_id]['pp shots'])
        if player_db[player_id]['ev shots'] != 0: player_db[player_id]['ev sv%'] = float(player_db[player_id]['ev shots'] - player_db[player_id]['ev goals against']) / float(player_db[player_id]['ev shots'])

    player_db[player_id]['adjusted sv%'] = player_db[player_id]['avg. shot difficulty'] * player_db[player_id]['sv%']
    player_db[player_id]['shot-goal difference'] = player_db[player_id]['avg. goal difficulty'] - player_db[player_id]['avg. shot difficulty']
    player_db[player_id]['adjusted pp sv%'] = player_db[player_id]['avg. pp shot difficulty'] * player_db[player_id]['pp sv%']
    player_db[player_id]['pp shot-goal difference'] = player_db[player_id]['avg. pp goal difficulty'] - player_db[player_id]['avg. pp shot difficulty']
    player_db[player_id]['adjusted ev sv%'] = player_db[player_id]['avg. ev shot difficulty'] * player_db[player_id]['ev sv%']
    player_db[player_id]['ev shot-goal difference'] = player_db[player_id]['avg. ev goal difficulty'] - player_db[player_id]['avg. ev shot difficulty']

player_db = pd.DataFrame.from_dict(player_db, orient='index')
player_db
#shot_details


,position,avg. shot difficulty,avg. goal difficulty,adjusted sv%,shot-goal difference,avg. pp shot difficulty,avg. pp goal difficulty,adjusted pp sv%,pp shot-goal difference,avg. ev shot difficulty,...,ev goals against,sv%,pp sv%,ev sv%,save locations,pp save locations,ev save locations,goal locations,pp goal locations,ev goal locations
"Saros, Juuse",G,0.929616,0.881878,0.837934,-0.047738,0.918793,0.885689,0.790432,-0.033104,0.931616,...,134,0.901376,0.860294,0.908967,"[[59, -26, right], [81, 8, right], [55, 30, ri...","[[34, 8, right], [-74, 17, left], [54, 14, rig...","[[59, -26, right], [81, 8, right], [55, 30, ri...","[[50, -16, right], [83, 0, right], [71, 7, rig...","[[83, 0, right], [82, 2, right], [87, -9, righ...","[[50, -16, right], [71, 7, right], [-77, 1, le..."
"Johansson, Jonas",G,0.925172,0.879065,0.818919,-0.046107,0.914152,0.866855,0.795312,-0.047296,0.926966,...,69,0.885154,0.870000,0.887622,"[[-75, -15, left], [-47, -25, left], [72, 39, ...","[[82, 1, right], [75, -16, right], [80, 8, rig...","[[-75, -15, left], [-47, -25, left], [72, 39, ...","[[81, 1, right], [-82, 4, left], [-73, -23, le...","[[-73, -23, left], [-78, 5, left], [47, 8, rig...","[[81, 1, right], [-82, 4, left], [-88, -1, lef..."
"Mrazek, Petr",G,0.926487,0.877889,0.836755,-0.048599,0.905391,0.868150,0.761963,-0.037241,0.929426,...,128,0.903148,0.841584,0.911724,"[[75, 21, right], [32, -8, right], [98, 12, ri...","[[59, -5, right], [60, -3, right], [62, 27, ri...","[[75, 21, right], [32, -8, right], [98, 12, ri...","[[83, -9, right], [-82, -19, left], [85, 5, ri...","[[-88, -4, left], [33, -4, right], [68, 2, rig...","[[83, -9, right], [-82, -19, left], [85, 5, ri..."
"Jarry, Tristan",G,0.928988,0.879448,0.834660,-0.049541,0.921079,0.872667,0.787104,-0.048413,0.930138,...,108,0.898462,0.854545,0.904846,"[[-55, -35, left], [-39, -27, left], [-36, -30...","[[-60, 25, left], [-78, -6, left], [-78, 2, le...","[[-55, -35, left], [-39, -27, left], [-36, -30...","[[83, 0, right], [-71, 11, left], [-67, 11, le...","[[83, -2, right], [-80, -5, left], [72, 21, ri...","[[83, 0, right], [-71, 11, left], [-67, 11, le..."
"Hill, Adin",G,0.931344,0.893113,0.842502,-0.038231,0.920822,0.925136,0.838360,0.004314,0.933109,...,77,0.904609,0.910448,0.903630,"[[74, 25, right], [87, -5, right], [68, 12, ri...","[[75, -7, right], [67, -6, right], [59, -28, r...","[[74, 25, right], [87, -5, right], [68, 12, ri...","[[-75, -1, left], [72, 6, right], [-86, -4, le...","[[-59, 2, left], [-51, 3, left], [-80, -4, lef...","[[-75, -1, left], [72, 6, right], [-86, -4, le..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Cooley, Devin",G,0.927392,0.882190,0.797868,-0.045202,0.934199,0.940700,0.896831,0.006501,0.926287,...,24,0.860335,0.960000,0.844156,"[[-62, -18, left], [-78, 1, left], [-76, 13, l...","[[-35, -5, left], [-55, -21, left], [-58, -8, ...","[[-62, -18, left], [-78, 1, left], [-76, 13, l...","[[80, 0, right], [-62, 10, left], [-31, -15, l...","[[-59, -8, left]]","[[80, 0, right], [-62, 10, left], [-31, -15, l..."
"Silovs, Arturs",G,0.932898,0.896574,0.817725,-0.036323,0.946684,0.927495,0.779622,-0.019189,0.929236,...,7,0.876543,0.823529,0.890625,"[[53, 36, right], [30, -35, right], [43, -5, r...","[[-34, -11, left], [-65, 20, left], [-80, 25, ...","[[53, 36, right], [30, -35, right], [43, -5, r...","[[78, 29, right], [82, -4, right], [75, -1, ri...","[[75, -1, right], [52, -11, right], [-39, -11,...","[[78, 29, right], [82, -4, right], [79, 14, ri..."
"Fedotov, Ivan",G,0.931939,0.895528,0.745551,-0.036411,0.961877,0.000000,0.961877,-0.961877,0.931328,...,10,0.800000,1.000000,0.795918,"[[-60, -39, left], [-66, -3, left], [-64, -22,...","[[-54, -23, left]]","[[-60, -39, left], [-66, -3, left], [-64, -22,...","[[-79, -6, left], [-67, 3, left], [86, -8, rig...",[],"[[-79, -6, left], [-67, 3, left], [86, -8, rig..."
"Subban, Malcolm",G,0.924922,0.953617,0.845643,0.028695,0.883222,0.942035,0.757048,0.058813,0.935347,...,2,0.

In [16]:
player_db = player_db.sort_values(by = ['sv%'], ascending=True)
player_db



,position,avg. shot difficulty,avg. goal difficulty,adjusted sv%,shot-goal difference,avg. pp shot difficulty,avg. pp goal difficulty,adjusted pp sv%,pp shot-goal difference,avg. ev shot difficulty,...,ev goals against,sv%,pp sv%,ev sv%,save locations,pp save locations,ev save locations,goal locations,pp goal locations,ev goal locations
"Appleby, Kenneth",G,0.897474,0.876358,0.673106,-0.021116,0.000000,0.000000,0.000000,0.000000,0.897474,...,2,0.750000,0.000000,0.750000,"[[-27, 22, left], [-55, -25, left], [-80, -3, ...",[],"[[-27, 22, left], [-55, -25, left], [-80, -3, ...","[[-61, 14, left], [-83, -5, left]]",[],"[[-61, 14, left], [-83, -5, left]]"
"Fedotov, Ivan",G,0.931939,0.895528,0.745551,-0.036411,0.961877,0.000000,0.961877,-0.961877,0.931328,...,10,0.800000,1.000000,0.795918,"[[-60, -39, left], [-66, -3, left], [-64, -22,...","[[-54, -23, left]]","[[-60, -39, left], [-66, -3, left], [-64, -22,...","[[-79, -6, left], [-67, 3, left], [86, -8, rig...",[],"[[-79, -6, left], [-67, 3, left], [86, -8, rig..."
"Sandstrom, Felix",G,0.921211,0.890794,0.749117,-0.030418,0.908199,0.891462,0.778457,-0.016737,0.925115,...,14,0.813187,0.857143,0.800000,"[[88, -9, right], [82, -12, right], [48, 38, r...","[[-73, -5, left], [-66, 19, left], [-83, 4, le...","[[88, -9, right], [82, -12, right], [48, 38, r...","[[-67, 16, left], [78, -15, right], [-86, -4, ...","[[46, 22, right], [-83, -4, left], [-68, 12, l...","[[-67, 16, left], [78, -15, right], [-86, -4, ..."
"Villalta, Matt",G,0.933568,0.883780,0.766860,-0.049788,0.936786,0.907226,0.702589,-0.029560,0.933032,...,4,0.821429,0.750000,0.833333,"[[-30, 28, left], [-84, -12, left], [-62, -24,...","[[-84, -12, left], [-62, -24, left], [-65, -13...","[[-30, 28, left], [-57, -32, left], [-74, -7, ...","[[-82, 10, left], [-78, -1, left], [-59, -17, ...","[[-82, 10, left]]","[[-78, -1, left], [-59, -17, left], [-83, -10,..."
"Chrona, Magnus",G,0.921894,0.864631,0.784906,-0.057263,0.925929,0.954204,0.827426,0.028275,0.920955,...,32,0.851406,0.893617,0.841584,"[[62, 17, right], [70, -12, right], [78, 0, ri...","[[-65, 26, left], [-82, 3, left], [-62, 13, le...","[[62, 17, right], [70, -12, right], [78, 0, ri...","[[84, 8, right], [-79, 2, left], [-86, 8, left...","[[-86, 8, left], [-57, 9, left], [-94, 23, lef...","[[84, 8, right], [-79, 2, left], [-86, 0, left..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Askarov, Yaroslav",G,0.938950,0.907744,0.885296,-0.031206,0.963741,0.000000,0.963741,-0.963741,0.938221,...,2,0.942857,1.000000,0.941176,"[[-72, 37, left], [-59, 12, left], [-79, -2, l...","[[80, 20, right]]","[[-72, 37, left], [-59, 12, left], [-79, -2, l...","[[-83, 7, left], [61, 21, right]]",[],"[[-83, 7, left], [61, 21, right]]"
"Domingue, Louis",G,0.942499,0.795167,0.906249,-0.147332,0.960011,0.000000,0.960011,-0.960011,0.941040,...,1,0.961538,1.000000,0.958333,"[[66, -17, right], [49, 14, right], [-52, 20, ...","[[-64, -16, left], [-69, 33, left]]","[[66, -17, right], [49, 14, right], [-52, 20, ...","[[-80, 0, left]]",[],"[[-80, 0, left]]"
"Romanov, Georgi",G,0.927742,0.778131,0.895751,-0.149611,0.000000,0.000000,0.000000,0.000000,0.927742,...,1,0.965517,0.000000,0.965517,"[[-65, 19, left], [-62, -31, left], [-72, 0, l...",[],"[[-65, 19, left], [-62, -31, left], [-72, 0, l...","[[-83, 4, left]]",[],"[[-83, 4, left]]"
"Murray, Matt",G,0.941315,0.000000,0.941315,-0.941315,0.906387,0.000000,0.906387,-0.906387,0.958778,...,0,1.000000,1.000000,1.000000,"[[-46, -15, left], [-75, -30, left], [-68, -26...","[[-28, -1, left], [-52, -27, left], [-83, 4, l...","[[-46, -15, left], [-75, -30, left], [-68, -26...",[],[],[]


In [15]:
#save data to pickle

player_db.to_pickle('2023-24_goaltender_shot_difficulty_metrics.pickle')

